In [1]:
!pip install turfpy
!pip install geojson

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [2]:
import json
import time
import socket
import uuid
import random
from datetime import datetime
import pytz
from turfpy.measurement import bbox, boolean_point_in_polygon
from geojson import Feature, Point, Polygon, MultiPolygon

# Cargar el GeoJSON de Medellín
with open('medellin.json', 'r') as f:
    medellin = json.load(f)
    
# Cargar el GeoJSON de las comunas y corregimientos
with open('barrios.json', 'r') as f:
    barrios = json.load(f)

box = bbox(medellin["features"][0]["geometry"])

# Definir las coordenadas aproximadas de Medellín
lat_range = (box[1], box[3])  # Latitud de Medellín
lng_range = (box[0], box[2])  # Longitud de Medellín


# Generar un punto aleatorio
def generate_point():
    while(True):
        lat_rand = round(random.uniform(*lat_range), 6)
        lng_rand = round(random.uniform(*lng_range), 6)
        point = Feature(geometry=Point([lng_rand, lat_rand]))
        polygons = [feature["geometry"]["coordinates"] for feature in barrios["features"]]
        multi_polygon = MultiPolygon(polygons)
        if boolean_point_in_polygon(point, multi_polygon):
            return lat_rand, lng_rand

# Generar datos simulados
def generate_event():
    lat, lng = generate_point()
    event = {
        "latitude": lat,
        "longitude": lng,
        "date": datetime.now(pytz.timezone('America/Bogota')).strftime("%d/%m/%Y %H:%M:%S"),
        "customer_id": random.randint(1000, 2000),
        "employee_id": random.randint(9000, 9040),
        "quantity_products": random.randint(10, 50),
        "order_id": str(uuid.uuid4()),
        # "commune": random.choice(comunas),
        # "neighborhood": random.choice(barrios),
        # "partition_date": datetime.now().strftime("%d%m%Y"),
        # "event_date": datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
        # "event_day": datetime.now().day,
        # "event_hour": datetime.now().hour,
        # "event_minute": datetime.now().minute,
        # "event_month": datetime.now().month,
        # "event_second": datetime.now().second,
        # "event_year": datetime.now().year
    }
    return event

def send_data(s, conn, data):
    conn.sendall(json.dumps(data).encode() + b'\n')
    print("enviado:", data)
    

def generate_data(host, port, interval):

    s = socket.socket()
    s.bind((host, port))
    s.listen(1)
    print(f"Listening on {host}:{port}")

    conn, addr = s.accept()
    print(f"Connection from {addr}")
    try:
        while True:
            event = generate_event()
            # Esperar 30 segundos antes de generar el siguiente evento
            send_data(s, conn, event)
            # Ejemplo de guardado en un archivo JSON
            with open('simulated_data.json', 'a') as file:
                file.write(json.dumps(event) + "\n")
            
            time.sleep(interval)
            
    except KeyboardInterrupt:
        print("Deteniendo el servidor...")
        conn.close()
        s.close()
        print("Conexión cerrada.")
    
    except BrokenPipeError:
        print("La conexión ha sido cerrada.")
        conn.close()
        s.close()



In [3]:
# generate_data(host, port, interval_in_seconds)
generate_data('localhost', 4224, 1)

Listening on localhost:4224
Connection from ('127.0.0.1', 35722)
enviado: {'latitude': 6.173934, 'longitude': -75.663362, 'date': '12/06/2024 12:28:42', 'customer_id': 1248, 'employee_id': 9019, 'quantity_products': 18, 'order_id': '06059f9e-43db-4e75-bf61-d50e2386ed7e'}
enviado: {'latitude': 6.248289, 'longitude': -75.70486, 'date': '12/06/2024 12:28:43', 'customer_id': 1636, 'employee_id': 9024, 'quantity_products': 12, 'order_id': '62adfc9b-3d4e-420c-9b65-9bfc22d762a6'}
enviado: {'latitude': 6.192264, 'longitude': -75.632434, 'date': '12/06/2024 12:28:45', 'customer_id': 1214, 'employee_id': 9034, 'quantity_products': 14, 'order_id': '2f7045e2-86aa-48fe-a691-73d96d251331'}
enviado: {'latitude': 6.244756, 'longitude': -75.679238, 'date': '12/06/2024 12:28:47', 'customer_id': 1536, 'employee_id': 9030, 'quantity_products': 14, 'order_id': '4910beee-d489-4cd8-9b1b-98ba918c2e35'}
enviado: {'latitude': 6.240215, 'longitude': -75.63636, 'date': '12/06/2024 12:28:49', 'customer_id': 1636, 